Importação das Bibliotecas

In [95]:
import pysolr
import json
import logging

Configuração do Logger

In [96]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Carregando .jsons

In [97]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Configuração da Conexão com o Solr

In [98]:
print("Conectando ao Solr...")
SOLR_URL_BASE = 'http://localhost:8983/solr/'
SOLR_CORE_NAME = 'documentos_ri'
SOLR_CONNECTION_STRING = f'{SOLR_URL_BASE}{SOLR_CORE_NAME}'

try:
    solr = pysolr.Solr(SOLR_CONNECTION_STRING, always_commit=True)
    solr.ping()
    logging.info(f"Conexão com o core '{SOLR_CORE_NAME}' bem-sucedida!")
except Exception as e:
    logging.error("Não foi possível conectar ao Solr. Verifique se o serviço está ativo.")
    logging.error(f"Detalhe: {e}")
    exit()


2025-07-22 05:51:49 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/admin/ping/?' (get) with body '' in 0.007 seconds, with status 200
2025-07-22 05:51:49 - INFO - Conexão com o core 'documentos_ri' bem-sucedida!


Conectando ao Solr...


Função que retorna Top N documentos de uma consulta no Solr

In [99]:
def obter_top_n_ids_solr(solr_client, texto_consulta, n=50):
    try:
        # 1. Constrói a string de busca a partir da consulta tokenizada
        texto_consulta = " ".join(consulta_tokenizada)
        query_string = f'texto_documento:({texto_consulta})'
        
        # 2. Executa a busca no Solr, que já retorna os resultados ordenados por score
        resultados_solr = solr_client.search(q=query_string, fl='id', rows=n)
        
        # 3. Retorna apenas os 'n' primeiros IDs da lista ordenada
        return [int(resultado['id']) for resultado in resultados_solr]

    except Exception as e:
            logging.error(f"Erro ao buscar a consulta '{texto_consulta[:30]}...': {e}")
            return [] # Retorna lista vazia em caso de erro na busca


Executar consulta e gerar os resultados top 50

In [100]:
# Escolha uma consulta para testar
logging.info("Executando consulta de exemplo...")
id_consulta_exemplo = "182"
texto_consulta_exemplo = map_consultas[id_consulta_exemplo]

logging.info(f"Texto da consulta (ID: {id_consulta_exemplo}): '{texto_consulta_exemplo}'")

# Tokeniza a consulta
consulta_tokenizada = texto_consulta_exemplo.lower().split()

# Top 50 documentos mais relevantes para a consulta
resultados_solr = obter_top_n_ids_solr(solr, consulta_tokenizada, n=50)

logging.info(f"Encontrados {len(resultados_solr)} resultados.")


2025-07-22 05:51:49 - INFO - Executando consulta de exemplo...
2025-07-22 05:51:49 - INFO - Texto da consulta (ID: 182): 'mutations sonic hedgehog genes affect developmental disorders'
2025-07-22 05:51:49 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/select/?q=texto_documento%3A%28mutations+sonic+hedgehog+genes+affect+developmental+disorders%29&fl=id&rows=50&wt=json' (get) with body '' in 0.007 seconds, with status 200
2025-07-22 05:51:49 - INFO - Encontrados 50 resultados.


Salvando resultados_solr como .json

In [101]:
try:
    with open('resultados_solr.json', 'w', encoding='utf-8') as f:
        json.dump(
            resultados_solr,
            f,
            indent=2,
            ensure_ascii=False
        )
    logging.info("Arquivo de resultados do Solr salvo com sucesso!")
except Exception as e:
    logging.error(f"ERRO ao salvar o arquivo JSON: {e}")
    

2025-07-22 05:51:49 - INFO - Arquivo de resultados do Solr salvo com sucesso!
